In [97]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql import DataFrame as SparkDataFrame
import plotly.express as px
import plotly.figure_factory as ff
from pyspark.sql.window import Window

In [2]:
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/15 23:39:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
train_raw: SparkDataFrame = spark.read.csv("../raw/train.csv", header=True)
test_raw: SparkDataFrame = spark.read.csv("../raw/test.csv", header=True)
store_raw: SparkDataFrame = spark.read.csv("../raw/store.csv", header=True)

In [4]:
train_raw.describe().toPandas()

23/05/15 23:39:10 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,summary,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,count,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209
1,mean,558.4297268309659,3.998340557348588,None,5773.818972305593,633.1459464082602,0.8301066939045958,0.3815145166824124,0.0,0.17864666946517382
2,stddev,321.9086511434528,1.9973909649405297,None,3849.9261752347525,464.4117338866323,0.3755392246931946,0.48575860487741956,0.0,0.38305636818092675
3,min,1,1,2013-01-01,0,0,0,0,0,0
4,max,999,7,2015-07-31,9999,999,1,1,c,1


In [5]:
train_raw.select(F.col("Store")).distinct().toPandas()

,Store
0,296
1,467
2,675
3,691
4,829
...,...
1110,119
1111,898
1112,1001
1113,438


In [6]:
store_raw.columns

['Store',
 'StoreType',
 'Assortment',
 'CompetitionDistance',
 'CompetitionOpenSinceMonth',
 'CompetitionOpenSinceYear',
 'Promo2',
 'Promo2SinceWeek',
 'Promo2SinceYear',
 'PromoInterval']

In [7]:
train_raw.columns

['Store',
 'DayOfWeek',
 'Date',
 'Sales',
 'Customers',
 'Open',
 'Promo',
 'StateHoliday',
 'SchoolHoliday']

In [13]:
train_raw.groupBy(F.col("Store")).count().orderBy(F.col("count")).groupBy(
    F.col("count")
).count().toPandas()

,count,count
0,941,1
1,942,934
2,758,180


In [16]:
train_raw.select(F.min(F.col("Date"))).toPandas()

,min(Date)
0,2013-01-01


In [17]:
train_raw.select(F.max(F.col("Date"))).toPandas()

,max(Date)
0,2015-07-31


In [18]:
store_raw.limit(10).toPandas()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270,9,2008,0,None,None,None
1,2,a,a,570,11,2007,1,13,2010,"Jan,Apr,Jul,Oct"
2,3,a,a,14130,12,2006,1,14,2011,"Jan,Apr,Jul,Oct"
3,4,c,c,620,9,2009,0,None,None,None
4,5,a,a,29910,4,2015,0,None,None,None
5,6,a,a,310,12,2013,0,None,None,None
6,7,a,c,24000,4,2013,0,None,None,None
7,8,a,a,7520,10,2014,0,None,None,None
8,9,a,c,2030,8,2000,0,None,None,None
9,10,a,a,3160,9,2009,0,None,None,None


In [19]:
store_raw.select(F.col("PromoInterval")).distinct().toPandas()

,PromoInterval
0,None
1,"Jan,Apr,Jul,Oct"
2,"Feb,May,Aug,Nov"
3,"Mar,Jun,Sept,Dec"


In [27]:
train_raw.groupby(F.col("Store")).agg(F.min(F.col("Date"))).orderBy(
    F.col("min(Date)")
).toPandas()

,Store,min(Date)
0,1,2013-01-01
1,10,2013-01-01
2,100,2013-01-01
3,1000,2013-01-01
4,1001,2013-01-01
...,...,...
1110,996,2013-01-01
1111,997,2013-01-01
1112,998,2013-01-01
1113,999,2013-01-01


In [30]:
data_counts: SparkDataFrame = train_raw.groupBy("Store").count().orderBy(F.col("count"))

In [35]:
low_count_stores: SparkDataFrame = data_counts.join(train_raw, on="Store").where(
    F.col("count") < 900
)

In [45]:
px.line(
    low_count_stores.where(F.col("Store") == 13)
    .select(F.col("Date"), F.col("Sales").cast("int"))
    .orderBy(F.col("Date"))
    .toPandas(),
    x="Date",
    y="Sales",
    title="Typical sales history for store with 758 days of history (Store 13)",
)

In [46]:
high_count_stores: SparkDataFrame = data_counts.join(train_raw, on="Store").where(
    F.col("count") > 941
)

In [47]:
high_count_stores.toPandas()

,Store,count,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,942,5,2015-07-31,5263,555,1,1,0,1
1,2,942,5,2015-07-31,6064,625,1,1,0,1
2,3,942,5,2015-07-31,8314,821,1,1,0,1
3,4,942,5,2015-07-31,13995,1498,1,1,0,1
4,5,942,5,2015-07-31,4822,559,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...
879823,1111,942,2,2013-01-01,0,0,0,0,a,1
879824,1112,942,2,2013-01-01,0,0,0,0,a,1
879825,1113,942,2,2013-01-01,0,0,0,0,a,1
879826,1114,942,2,2013-01-01,0,0,0,0,a,1


In [48]:
px.line(
    high_count_stores.where(F.col("Store") == 3)
    .select(F.col("Date"), F.col("Sales").cast("int"))
    .orderBy(F.col("Date"))
    .toPandas(),
    x="Date",
    y="Sales",
    title="Typical sales history for store with 942 days of history (Store 3)",
)

In [64]:
px.histogram(
    train_raw.filter((F.col("Store") == 13) & (F.year(F.col("Date")) < 2015))
    .filter(F.col("Sales") != 0)
    .select(F.col("Sales").astype("int"))
    .toPandas()
)

In [81]:
pre_refurb: list = (
    train_raw.filter(F.col("Store") == 13)
    .filter(F.year(F.col("Date")) < 2015)
    .filter(F.col("Sales") != 0)
    .select(F.col("Sales").astype("int"))
    .toPandas()["Sales"]
    .to_list()
)
post_refurb: list = (
    train_raw.filter(F.col("Store") == 13)
    .filter(F.year(F.col("Date")) >= 2015)
    .filter(F.col("Sales") != 0)
    .select(F.col("Sales").astype("int"))
    .toPandas()["Sales"]
    .to_list()
)

hist_data = [pre_refurb, post_refurb]
group_labels = ["pre", "post"]

fig = ff.create_distplot(hist_data, group_labels, bin_size=1000)
fig.update_layout(
    title=dict(text="Sales density pre and post shutdown period (Store 13)")
)
fig.update_yaxes(visible=False)
fig.show()

In [74]:
min(pre_refurb)

2210

In [82]:
store_raw.limit(10).toPandas()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270,9,2008,0,None,None,None
1,2,a,a,570,11,2007,1,13,2010,"Jan,Apr,Jul,Oct"
2,3,a,a,14130,12,2006,1,14,2011,"Jan,Apr,Jul,Oct"
3,4,c,c,620,9,2009,0,None,None,None
4,5,a,a,29910,4,2015,0,None,None,None
5,6,a,a,310,12,2013,0,None,None,None
6,7,a,c,24000,4,2013,0,None,None,None
7,8,a,a,7520,10,2014,0,None,None,None
8,9,a,c,2030,8,2000,0,None,None,None
9,10,a,a,3160,9,2009,0,None,None,None


In [84]:
train_raw.limit(10).toPandas()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1
5,6,5,2015-07-31,5651,589,1,1,0,1
6,7,5,2015-07-31,15344,1414,1,1,0,1
7,8,5,2015-07-31,8492,833,1,1,0,1
8,9,5,2015-07-31,8565,687,1,1,0,1
9,10,5,2015-07-31,7185,681,1,1,0,1


In [95]:
px.scatter(
    train_raw.filter(F.col("Store") == 3)
    .filter(F.col("Sales") != 0)
    .select(F.col("Sales").astype("int"), F.col("Customers").astype("int"))
    .toPandas(),
    x="Customers",
    y="Sales",
)

In [94]:
px.line(
    train_raw.filter(F.col("Store") == 3)
    .select(F.col("Customers").astype("int"), F.col("Date"))
    .toPandas(),
    x="Date",
    y="Customers",
)

In [96]:
store_3: SparkDataFrame = train_raw.filter(F.col("Store") == 3)

In [103]:
windowSpec = Window.partitionBy(F.col("Store")).orderBy("Date")

In [126]:
px.scatter(
    (
        store_3.withColumn(
            "Yesterday's Customers", F.lag("Customers", 1).over(windowSpec)
        )
        .filter(F.col("Yesterday's Customers") != 0)
        .filter(F.col("Yesterday's Customers").isNotNull())
        .filter(F.col("Sales") != 0)
        .select(
            F.col("Sales").astype("int"), F.col("Yesterday's Customers").astype("int")
        )
    ).toPandas(),
    x="Yesterday's Customers",
    y="Sales",
    title="Correlation between sales today and yesterday's customers (Store 3)",
    trendline="ols",
)

In [127]:
store_raw.limit(10).toPandas()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270,9,2008,0,None,None,None
1,2,a,a,570,11,2007,1,13,2010,"Jan,Apr,Jul,Oct"
2,3,a,a,14130,12,2006,1,14,2011,"Jan,Apr,Jul,Oct"
3,4,c,c,620,9,2009,0,None,None,None
4,5,a,a,29910,4,2015,0,None,None,None
5,6,a,a,310,12,2013,0,None,None,None
6,7,a,c,24000,4,2013,0,None,None,None
7,8,a,a,7520,10,2014,0,None,None,None
8,9,a,c,2030,8,2000,0,None,None,None
9,10,a,a,3160,9,2009,0,None,None,None


In [135]:
joined_raw: SparkDataFrame = train_raw.join(store_raw, ["Store"])

In [141]:
joined_raw.filter(F.col("Assortment") == "c").toPandas()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620,9,2009,0,None,None,None
1,7,5,2015-07-31,15344,1414,1,1,0,1,a,c,24000,4,2013,0,None,None,None
2,9,5,2015-07-31,8565,687,1,1,0,1,a,c,2030,8,2000,0,None,None,None
3,11,5,2015-07-31,10457,1236,1,1,0,1,a,c,960,11,2011,1,1,2012,"Jan,Apr,Jul,Oct"
4,12,5,2015-07-31,8959,962,1,1,0,1,a,c,1070,None,None,1,13,2010,"Jan,Apr,Jul,Oct"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471465,1110,2,2013-01-01,0,0,0,0,a,1,c,c,900,9,2010,0,None,None,None
471466,1112,2,2013-01-01,0,0,0,0,a,1,c,c,1880,4,2006,0,None,None,None
471467,1113,2,2013-01-01,0,0,0,0,a,1,a,c,9260,None,None,0,None,None,None
471468,1114,2,2013-01-01,0,0,0,0,a,1,a,c,870,None,None,0,None,None,None


In [137]:
store_3_joined: SparkDataFrame = joined_raw.filter(F.col("Store") == 3)

In [139]:
store_259_joined: SparkDataFrame = joined_raw.filter(F.col("Store") == 259)

In [144]:
store_4_joined: SparkDataFrame = joined_raw.filter(F.col("Store") == 4)

In [152]:
hist_data = [
    df.filter(F.col("Sales") != 0)
    .select(F.col("Sales").astype("int"))
    .toPandas()["Sales"]
    .to_list()
    for df in [store_3_joined, store_259_joined, store_4_joined]
]
group_labels = [
    "Assortment A (Store 3)",
    "Assortment B (Store 259)",
    "Assortment C (Store 4)",
]

fig = ff.create_distplot(hist_data, group_labels, bin_size=1000)
fig.update_layout(title=dict(text="Sales density by assortment in store"))
fig.update_yaxes(visible=False)
fig.show()

$$h_{loss}$$